In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import QuarterEnd
import pytz
import time
import os

In [55]:
#1) Read CSV

In [47]:
df = pd.read_csv("step_2_table.csv")
df = df[["quarter","ticker","cik"]]
df.head()

,quarter,ticker,cik
0,Q1-19,A,1090872
1,Q1-19,AAL,6201
2,Q1-19,AAP,1158449
3,Q1-19,AAPL,320193
4,Q1-19,ABBV,1551152


In [54]:
#2) get sector and industry
# unique ids to not get blocked

In [57]:
unique_tickers = df['ticker'].unique()
unique_tickers.shape

(632,)

In [62]:
"""tickers = df['ticker'].unique()

# Function to split list into batches
def batch_list(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Storage for metadata
sector_industry_map = {}

In [65]:
for batch in batch_list(tickers, 100):  # adjust size if needed
    print(f"Processing batch: {batch[0]} to {batch[-1]}")
    for ticker in batch:
        try:
            info = yf.Ticker(ticker).info
            sector_industry_map[ticker] = {
                'sector': info.get('sector', None),
                'industry': info.get('industry', None)
            }
        except Exception as e:
            print(f"Failed to fetch {ticker}: {e}")
            sector_industry_map[ticker] = {'sector': None, 'industry': None}
        time.sleep(0.1)  # slight pause to be polite to Yahoo servers

Processing batch: A to COST
Processing batch: COTY to HRB
Processing batch: HRL to NOC
Processing batch: NOV to UDR
Processing batch: UHS to FDS
Processing batch: SEDG to KSU
Failed to fetch HCP: HTTP Error 404: 
Processing batch: LLL to SBNY


In [69]:
#map back
df['sector'] = df['ticker'].map(lambda t: sector_industry_map.get(t, {}).get('sector'))
df['industry'] = df['ticker'].map(lambda t: sector_industry_map.get(t, {}).get('industry'))
df.head()

,quarter,ticker,cik,quarter_end,sector,industry
0,Q1-19,A,1090872,2019-03-31,Healthcare,Diagnostics & Research
1,Q1-19,AAL,6201,2019-03-31,Industrials,Airlines
2,Q1-19,AAP,1158449,2019-03-31,Consumer Cyclical,Auto Parts
3,Q1-19,AAPL,320193,2019-03-31,Technology,Consumer Electronics
4,Q1-19,ABBV,1551152,2019-03-31,Healthcare,Drug Manufacturers - General


In [111]:
#saving just in case
industry_df = pd.DataFrame.from_dict(sector_industry_map, orient='index')
industry_df.reset_index(inplace=True)
industry_df.columns = ['ticker', 'sector', 'industry']
industry_df.to_csv('ticker_sector_industry.csv', index=False)

In [112]:
industry_df = pd.read_csv('ticker_sector_industry.csv')
industry_df.replace('None', pd.NA, inplace=True)

In [113]:
missing_industry = industry_df[industry_df['sector'].isna() | industry_df['industry'].isna()]

In [114]:
len(missing_industry)

69

In [115]:
#trying wiki

In [119]:
df_wiki = pd.read_csv("step_2_table.csv")
df_wiki = df_wiki[["ticker", "company_name"]]
df_wiki = df_wiki.drop_duplicates(subset=['ticker', 'company_name'])

In [120]:
wiki_df = missing_industry.merge(df_wiki, on='ticker', how='left')

In [123]:
import wikipedia

def get_wikipedia_industry(name):
    try:
        # Search using company name
        summary = wikipedia.summary(name, auto_suggest=False)
        # Try to extract industry manually from summary (very naive)
        for line in summary.split('.'):
            if 'industry' in line.lower():
                return line.strip()
        return None
    except Exception as e:
        print(f"Wiki lookup failed for {name}: {e}")
        return None

In [ ]:
#3) get quarter end exact date for price

In [126]:
df = pd.read_csv("step_2_table.csv")
df = df[["quarter","ticker","cik"]]
df.head()

,quarter,ticker,cik
0,Q1-19,A,1090872
1,Q1-19,AAL,6201
2,Q1-19,AAP,1158449
3,Q1-19,AAPL,320193
4,Q1-19,ABBV,1551152


In [127]:
def get_quarter_end(quarter_str):
    q, y = quarter_str.split('-')
    year = int('20' + y)
    quarter_end_dates = {
        'Q1': datetime(year, 3, 31),
        'Q2': datetime(year, 6, 30),
        'Q3': datetime(year, 9, 30),
        'Q4': datetime(year, 12, 31),
    }
    return quarter_end_dates[q]

In [128]:
get_quarter_end("Q1-19")

datetime.datetime(2019, 3, 31, 0, 0)

In [129]:
df['quarter_end'] = df['quarter'].apply(get_quarter_end)

In [130]:
df.shape

(11596, 4)

In [136]:
df

,quarter,ticker,cik,quarter_end
0,Q1-19,A,1090872,2019-03-31
1,Q1-19,AAL,6201,2019-03-31
2,Q1-19,AAP,1158449,2019-03-31
3,Q1-19,AAPL,320193,2019-03-31
4,Q1-19,ABBV,1551152,2019-03-31
...,...,...,...,...
11591,Q2-24,CTLT,1596783,2024-06-30
11592,Q2-24,MRO,101778,2024-06-30
11593,Q2-24,WRK,1732845,2024-06-30
11594,Q3-24,CTLT,1596783,2024-09-30


In [140]:
master_filename = "quarterly_prices_all.csv"

In [141]:
# Load existing results if continuing from previous run
if os.path.exists(master_filename):
    all_results = pd.read_csv(master_filename, parse_dates=['quarter_end'])
    completed_quarters = set(all_results['quarter'].unique())
    print(f"📄 Loaded existing results for {len(completed_quarters)} quarters.")
else:
    all_results = pd.DataFrame()
    completed_quarters = set()

In [142]:
# Load existing results if continuing from previous run
if os.path.exists(master_filename):
    all_results = pd.read_csv(master_filename, parse_dates=['quarter_end'])
    completed_quarters = set(all_results['quarter'].unique())
    print(f"📄 Loaded existing results for {len(completed_quarters)} quarters.")
else:
    all_results = pd.DataFrame()
    completed_quarters = set()

# --- Loop through each quarter ---
for q in sorted(df['quarter'].unique()):
    if q in completed_quarters:
        print(f"✅ {q} already processed — skipping.")
        continue

    q_end = get_quarter_end(q)
    tickers = df[df['quarter'] == q]['ticker'].unique().tolist()
    start_date = q_end - timedelta(days=7)
    end_date = q_end + timedelta(days=1)

    print(f"\n🔄 Fetching {len(tickers)} tickers for {q} (ending {q_end.date()})")

    try:
        data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=False)
    except Exception as e:
        print(f"❌ Error downloading for {q}: {e}")
        continue

    quarter_results = []

    for ticker in tickers:
        try:
            if len(tickers) == 1:
                closes = data['Close'].dropna()
            else:
                closes = data[ticker]['Close'].dropna()

            if not closes.empty:
                last_close_date = closes.index[closes.index <= pd.Timestamp(q_end)].max()
                last_price = closes.loc[last_close_date]
            else:
                last_price = float('nan')
        except Exception as e:
            print(f"⚠️  Error processing {ticker}: {e}")
            last_price = float('nan')

        quarter_results.append({
            'ticker': ticker,
            'quarter': q,
            'quarter_end': q_end,
            'Q_End_Price': last_price
        })

    # --- Append and overwrite master CSV ---
    new_df = pd.DataFrame(quarter_results)
    all_results = pd.concat([all_results, new_df], ignore_index=True)
    all_results.to_csv(master_filename, index=False)
    print(f"💾 Appended {len(new_df)} results to {master_filename}")


🔄 Fetching 505 tickers for Q1-19 (ending 2019-03-31)



66 Failed downloads:
['NBL', 'CELG', 'FISV', 'CXO', 'DISCK', 'UTX', 'ABMD', 'BLL', 'ETFC', 'FBHS', 'WLTW', 'WRK', 'FLT', 'WCG', 'ATVI', 'RTN', 'FRC', 'AGN', 'PBCT', 'BHGE', 'XLNX', 'VIAB', 'BBT', 'ANTM', 'DISH', 'TSS', 'RE', 'COG', 'HRS', 'CTXS', 'VAR', 'DWDP', 'GPS', 'MXIM', 'APC', 'JEC', 'TWTR', 'ALXN', 'ADS', 'CERN', 'FB', 'MRO', 'ABC', 'ARNC', 'SIVB', 'TMK', 'TIF', 'FLIR', 'HCP', 'HFC', 'DRE', 'CBS', 'KSU', 'PKI', 'SYMC', 'CTL', 'MYL', 'NLSN', 'DISCA', 'LLL', 'XEC', 'RHT', 'PXD']: YFTzMissingError('possibly delisted; no timezone found')
['INFO', 'STI', 'LB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-03-24 00:00:00 -> 2019-04-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1553400000, endDate = 1554091200")')


💾 Appended 505 results to quarterly_prices_all.csv

🔄 Fetching 505 tickers for Q1-20 (ending 2020-03-31)



51 Failed downloads:
['NBL', 'FISV', 'UTX', 'DISCK', 'CXO', 'BLL', 'ETFC', 'FLT', 'FBHS', 'WLTW', 'WRK', 'ATVI', 'AGN', 'RTN', 'FRC', 'ABMD', 'PBCT', 'XLNX', 'ANTM', 'DISH', 'RE', 'CTXS', 'VAR', 'COG', 'MXIM', 'TWTR', 'ALXN', 'ADS', 'GPS', 'CERN', 'MRO', 'FB', 'ARNC', 'SIVB', 'TIF', 'ABC', 'FLIR', 'PEAK', 'MYL', 'HFC', 'DRE', 'KSU', 'PKI', 'NLSN', 'NLOK', 'CTL', 'DISCA', 'VIAC', 'PXD']: YFTzMissingError('possibly delisted; no timezone found')
['INFO', 'LB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-03-24 00:00:00 -> 2020-04-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1585022400, endDate = 1585713600")')


💾 Appended 505 results to quarterly_prices_all.csv

🔄 Fetching 505 tickers for Q1-21 (ending 2021-03-31)


...

💾 Appended 505 results to quarterly_prices_all.csv

🔄 Fetching 505 tickers for Q4-21 (ending 2021-12-31)



35 Failed downloads:
['FISV', 'DISCK', 'ABMD', 'BLL', 'WLTW', 'FBHS', 'FLT', 'WRK', 'ATVI', 'FRC', 'ANTM', 'DISH', 'RE', 'CTXS', 'GPS', 'PBCT', 'XLNX', 'TWTR', 'CERN', 'MRO', 'FB', 'ABC', 'SIVB', 'PEAK', 'DRE', 'CDAY', 'NLOK', 'PKI', 'NLSN', 'DISCA', 'CTLT', 'VIAC', 'PXD']: YFTzMissingError('possibly delisted; no timezone found')
['SBNY', 'INFO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-12-24 00:00:00 -> 2022-01-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1640322000, endDate = 1641013200")')


💾 Appended 505 results to quarterly_prices_all.csv

🔄 Fetching 503 tickers for Q4-22 (ending 2022-12-31)



16 Failed downloads:
['FISV', 'FLT', 'WRK', 'ATVI', 'FRC', 'DISH', 'RE', 'ABC', 'MRO', 'PEAK', 'CDAY', 'PKI', 'CTLT', 'SIVB', 'PXD']: YFTzMissingError('possibly delisted; no timezone found')
['SBNY']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-12-24 00:00:00 -> 2023-01-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1671858000, endDate = 1672549200")')


💾 Appended 503 results to quarterly_prices_all.csv

🔄 Fetching 503 tickers for Q4-24 (ending 2024-12-31)
💾 Appended 503 results to quarterly_prices_all.csv


In [4]:
df = pd.read_csv("quarterly_prices_all.csv")

In [5]:
df.head()

,ticker,quarter,quarter_end,Q_End_Price
0,A,Q1-19,31/03/2019,76.857788
1,AAL,Q1-19,31/03/2019,31.339823
2,AAP,Q1-19,31/03/2019,152.464035
3,AAPL,Q1-19,31/03/2019,45.441719
4,ABBV,Q1-19,31/03/2019,61.163391


In [6]:
import pandas as pd
import yfinance as yf
from functools import lru_cache
import swifter  # Optional but faster if installed
import os

/Users/chloecurtis/.pyenv/versions/3.12.9/envs/corporate-sentiment-tracker/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# ---------- 1. Cache yf.Ticker ----------
@lru_cache(maxsize=1000)
def get_stock(ticker):
    return yf.Ticker(ticker)

# ---------- 2. Combined earnings + previous close logic ----------
def process_row(ticker, quarter_end):
    try:
        quarter_end = pd.to_datetime(quarter_end, dayfirst=True)
        stock = get_stock(ticker)

        earnings_df = stock.get_earnings_dates(limit=12)
        if earnings_df is None or earnings_df.empty:
            return None, None

        earnings_df = earnings_df.copy()
        earnings_df.index = pd.to_datetime(earnings_df.index, errors='coerce')
        earnings_df.dropna(inplace=True)

        valid_dates = earnings_df[earnings_df.index <= quarter_end]
        if valid_dates.empty:
            return None, None

        earnings_date = valid_dates.index.max()

        prev_day = earnings_date - pd.Timedelta(days=1)
        start_date = prev_day - pd.Timedelta(days=5)
        history = stock.history(start=start_date.strftime('%Y-%m-%d'), end=earnings_date.strftime('%Y-%m-%d'))

        for i in range(5):
            lookup_day = prev_day - pd.Timedelta(days=i)
            if lookup_day in history.index:
                return earnings_date, history.loc[lookup_day, 'Close']

    except Exception as e:
        print(f"Error for {ticker}: {e}")

    return None, None

In [12]:
print(process_row("AAPL", "31/03/2019"))  # Should return a real date + float close
print(process_row("ZZZZ", "31/03/2019"))

Error for AAPL: 'NoneType' object is not subscriptable
(None, None)
Error for ZZZZ: 'NoneType' object is not subscriptable
(None, None)


In [ ]:
# ---------- 3. Main loop ----------
df['quarter_end'] = pd.to_datetime(df['quarter_end'], dayfirst=True)
output_file = "earnings_with_prices.csv"
df_master = pd.DataFrame()
batch_size = 500

for i in range(0, len(df), batch_size):
    print(f"Processing rows {i} to {i + batch_size - 1}")
    df_batch = df.iloc[i:i+batch_size].copy()

    df_batch[['earnings_date', 'prev_day_close']] = df_batch.swifter.apply(
        lambda row: pd.Series(process_row(row['ticker'], row['quarter_end'])),
        axis=1
    )

    df_master = pd.concat([df_master, df_batch], ignore_index=True)

    # Save after each batch (overwrite)
    df_master.to_csv(output_file, index=False)
    print(f"Batch saved: {output_file}")


Processing rows 0 to 499


Pandas Apply:   1%|▏                            | 3/500 [00:00<00:35, 14.17it/s]

Error for A: 'NoneType' object is not subscriptable
Error for AAL: 'NoneType' object is not subscriptable


Pandas Apply:   1%|▎                            | 5/500 [00:00<00:42, 11.72it/s]

Error for AAP: 'NoneType' object is not subscriptable
Error for AAPL: 'NoneType' object is not subscriptable
Error for ABBV: 'NoneType' object is not subscriptable


...

Error for STT: 'NoneType' object is not subscriptable
Error for STX: 'NoneType' object is not subscriptable


Pandas Apply:  75%|████████████████████▎      | 376/500 [00:36<00:12, 10.01it/s]

Error for STZ: 'NoneType' object is not subscriptable
Error for SWK: 'NoneType' object is not subscriptable
Error for SWKS: 'NoneType' object is not subscriptable


Pandas Apply:  76%|████████████████████▌      | 380/500 [00:37<00:11, 10.10it/s]

Error for SYF: 'NoneType' object is not subscriptable
Error for SYK: 'NoneType' object is not subscriptable
Error for SYY: 'NoneType' object is not subscriptable


In [13]:
import yfinance as yf
import pandas as pd

ticker = "AAPL"
stock = yf.Ticker(ticker)
earnings_df = stock.get_earnings_dates(limit=12)

print("earnings_df type:", type(earnings_df))
print("earnings_df content:")
print(earnings_df)

print("earnings_df index:")
print(earnings_df.index)
print("earnings_df.index type:", type(earnings_df.index))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Add columns for earnings date and time
df["EarningsDate"] = None
df["EarningsTime"] = None

for idx, row in df.iterrows():
    ticker = row["ticker"]
    quarter = row["quarter"]

    try:
        start_date, end_date = get_quarter_date_range(quarter)

        # Fetch earnings dates
        ticker_obj = yf.Ticker(ticker)
        earnings_df = ticker_obj.get_earnings_dates(limit=40)

        if earnings_df is not None and not earnings_df.empty:
            # Localize range to match earnings_df index timezone
            tz = earnings_df.index.tz or pytz.timezone("America/New_York")
            start_date = tz.localize(start_date)
            end_date = tz.localize(end_date)

            # Filter earnings within the range
            filtered = earnings_df[(earnings_df.index >= start_date) & (earnings_df.index <= end_date)]

            if not filtered.empty:
                full_dt = filtered.index[0]
                df.at[idx, "EarningsDate"] = full_dt  # full timestamp
                df.at[idx, "EarningsTime"] = full_dt.time()  # just time (HH:MM:SS)
            else:
                print(f"No earnings data for {ticker} in {quarter}")
    except Exception as e:
        print(f"Error fetching earnings for {ticker}: {e}")

In [92]:
df = pd.DataFrame(data, columns=["Quarter", "Ticker", "CIK", "Company"])

In [93]:
def get_close_price(ticker, earnings_dt, session):
    try:
        ticker_obj = yf.Ticker(ticker)
        earnings_dt = pd.Timestamp(earnings_dt)

        if session == "BMO":
            price_date = earnings_dt
        elif session == "AMC":
            price_date = earnings_dt + pd.Timedelta(days=1)
        else:
            return None
            
        price_data = ticker_obj.history(start=price_date, end=price_date + pd.Timedelta(days=2))

        if price_date in price_data.index:
            return price_data.loc[price_date]["Close"]
        else:
            return price_data["Close"].iloc[0] if not price_data.empty else None

        # Calculate CoD (Change on Day %)
        try:
                if session == "BMO":
                    prev_day = full_dt.date() - pd.Timedelta(days=1)
                    price_data = ticker_obj.history(start=prev_day, end=full_dt.date() + pd.Timedelta(days=1))
                    if prev_day in price_data.index and full_dt.date() in price_data.index:
                        prev_close = price_data.loc[prev_day]["Close"]
                        today_close = price_data.loc[full_dt.date()]["Close"]
                        cod = ((today_close - prev_close) / prev_close) * 100                            
                        df.at[idx, "CoD"] = round(cod, 2)
                elif session == "AMC":
                    next_day = full_dt.date() + pd.Timedelta(days=1)
                    price_data = ticker_obj.history(start=full_dt.date(), end=next_day + pd.Timedelta(days=1))
                    if full_dt.date() in price_data.index and next_day in price_data.index:
                        today_close = price_data.loc[full_dt.date()]["Close"]
                        next_close = price_data.loc[next_day]["Close"]
                        cod = ((next_close - today_close) / today_close) * 100
                        df.at[idx, "CoD"] = round(cod, 2)
                    
        except Exception as e:
            print(f"Error getting close price for {ticker} on {earnings_dt}: {e}")

# Initialize columns
df["EarningsDate"] = None
df["EarningsTime"] = None
df["EarningsSession"] = None
df["ClosePriceEarnings"] = None
df["Sector"] = None
df["CoD"] = None

# Process each row
for idx, row in df.iterrows():
    ticker = row["Ticker"]
    quarter = row["Quarter"]

    try:
        start_date, end_date = get_quarter_date_range(quarter)
        ticker_obj = yf.Ticker(ticker)

        earnings_df = ticker_obj.get_earnings_dates(limit=30)

        if earnings_df is not None and not earnings_df.empty:
            tz = earnings_df.index.tz or pytz.timezone("America/New_York")
            start_date = tz.localize(start_date)
            end_date = tz.localize(end_date)

            filtered = earnings_df[(earnings_df.index >= start_date) & (earnings_df.index <= end_date)]

            if not filtered.empty:
                full_dt = filtered.index[0]
                df.at[idx, "EarningsDate"] = full_dt.date()
                df.at[idx, "EarningsTime"] = full_dt.time()

                # Determine session (AMC or BMO)
                session = "AMC" if full_dt.time() >= datetime.strptime("15:00:00", "%H:%M:%S").time() else "BMO"
                df.at[idx, "EarningsSession"] = session

                # Get close price
                close_price = get_close_price(ticker, full_dt.date(), session)
                df.at[idx, "ClosePriceEarnings"] = close_price

        # Get sector info
        info = ticker_obj.info
        df.at[idx, "Sector"] = info.get("sector", "N/A")
        
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

df

SyntaxError: expected 'except' or 'finally' block (828948841.py, line 43)

In [80]:
from datetime import time

# Define cutoff time
cutoff = time(12, 0)  # 3:00 PM

df["EarningsSession"] = df["EarningsTime"].apply(
    lambda t: "AMC" if pd.notnull(t) and t > cutoff else ("BMO" if pd.notnull(t) else None)
)

In [81]:
df

,Quarter,Ticker,CIK,Company,EarningsDate,EarningsTime,EarningsSession,ClosePriceEarnings,Sector,CoD
0,Q1-19,A,0001090872,"AGILENT TECHNOLOGIES, INC.",2019-02-20,16:05:00,AMC,74.476906,Healthcare,None
1,Q1-19,AAL,0000006201,American Airlines Group Inc.,2019-01-24,07:30:00,BMO,33.124378,Industrials,None
2,Q1-19,AAP,0001158449,ADVANCE AUTO PARTS INC,2019-02-19,06:30:00,BMO,149.37854,Consumer Cyclical,None
3,Q1-19,AAPL,0000320193,Apple Inc.,2019-01-29,16:30:00,AMC,39.363914,Technology,None
4,Q1-19,ABBV,0001551152,AbbVie Inc.,2019-01-25,07:47:00,BMO,61.125439,Healthcare,None


KeyError: 'ticker'

In [16]:
def get_quarter_date_range(qtr_str):
    qtr, yr = qtr_str.split('-')
    year_full = int("20" + yr) if len(yr) == 2 else int(yr)
    start_date = pd.to_datetime(f"{year_full}-{quarter_months[qtr][0]}")
    end_date = pd.to_datetime(f"{year_full}-{quarter_months[qtr][1]}")
    return start_date, end_date

def get_earnings_date(ticker, start_date, end_date):
    try:
        ticker_obj = yf.Ticker(ticker)
        # Try to get earnings dates DataFrame
        earnings_dates_df = ticker_obj.get_earnings_dates(limit=10)  # last 10 earnings dates

        if earnings_dates_df is None or earnings_dates_df.empty:
            return None

        # Inspect columns
        # Typical columns: 'EPS Estimate', 'Reported EPS', 'Surprise(%)', 'EPS Report Date'
        # Look for date columns:
        date_col_candidates = [col for col in earnings_dates_df.columns if 'date' in col.lower()]
        if not date_col_candidates:
            return None
        
        date_col = date_col_candidates[0]
        
        earnings_dates_df[date_col] = pd.to_datetime(earnings_dates_df[date_col], errors='coerce')
        filtered = earnings_dates_df[
            (earnings_dates_df[date_col] >= start_date) & 
            (earnings_dates_df[date_col] <= end_date)
        ]

        if filtered.empty:
            return None
        
        # Return earliest earnings date in that quarter
        return filtered[date_col].iloc[0].date()
    except Exception as e:
        print(f"Error fetching earnings for {ticker}: {e}")
        return None

earnings_dates = []
for idx, row in df.iterrows():
    start_date, end_date = get_quarter_date_range(row['Quarter'])
    earnings_date = get_earnings_date(row['Ticker'], start_date, end_date)
    earnings_dates.append(earnings_date)

df['EarningsDate'] = earnings_dates
print(df)

  Quarter Ticker         CIK                       Company EarningsDate
0   Q1-19      A  0001090872    AGILENT TECHNOLOGIES, INC.         None
1   Q1-19    AAL  0000006201  American Airlines Group Inc.         None
2   Q1-19    AAP  0001158449        ADVANCE AUTO PARTS INC         None
3   Q1-19   AAPL  0000320193                    Apple Inc.         None
4   Q1-19   ABBV  0001551152                   AbbVie Inc.         None


In [13]:
df

,Quarter,Ticker,CIK,Company,EarningsDateRange,EarningsDate
0,Q1-19,A,0001090872,"AGILENT TECHNOLOGIES, INC.",2019-01-01 to 2019-03-31,None
1,Q1-19,AAL,0000006201,American Airlines Group Inc.,2019-01-01 to 2019-03-31,None
2,Q1-19,AAP,0001158449,ADVANCE AUTO PARTS INC,2019-01-01 to 2019-03-31,None
3,Q1-19,AAPL,0000320193,Apple Inc.,2019-01-01 to 2019-03-31,None
4,Q1-19,ABBV,0001551152,AbbVie Inc.,2019-01-01 to 2019-03-31,None


In [26]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Sample data (replace with your full DataFrame)
data = {
    "Quarter": ["Q1-19", "Q1-19", "Q2-19"],
    "Ticker": ["AAPL", "AAPL", "AAPL"],
}
df = pd.DataFrame(data)

# Helper to convert quarter string to date range
def quarter_to_dates(quarter_str):
    q, y = quarter_str.split("-")
    year = int("20" + y)  # convert '19' -> 2019
    if q == "Q1":
        return (datetime(year, 1, 1), datetime(year, 3, 31))
    elif q == "Q2":
        return (datetime(year, 4, 1), datetime(year, 6, 30))
    elif q == "Q3":
        return (datetime(year, 7, 1), datetime(year, 9, 30))
    elif q == "Q4":
        return (datetime(year, 10, 1), datetime(year, 12, 31))

# Add a new column for earnings date(s)
df["EarningsDate"] = None

for idx, row in df.iterrows():
    ticker = row["Ticker"]
    quarter = row["Quarter"]
    start_date, end_date = quarter_to_dates(quarter)

    ticker_obj = yf.Ticker(ticker)

    try:
        earnings_df = ticker_obj.get_earnings_dates(limit=30)
        if earnings_df.empty:
            print(f"No earnings data for {ticker}")
            continue

        # earnings_df.index contains earnings dates as pandas Timestamp
        # Filter rows where the earnings date falls within quarter range
        mask = (earnings_df.index >= pd.Timestamp(start_date)) & (earnings_df.index <= pd.Timestamp(end_date))
        filtered = earnings_df.loc[mask]

        if not filtered.empty:
            # Store earnings dates as string joined by comma (if multiple)
            earnings_dates_str = ", ".join(date.strftime("%Y-%m-%d") for date in filtered.index)
            df.at[idx, "EarningsDate"] = earnings_dates_str
        else:
            df.at[idx, "EarningsDate"] = None
            print(f"No earnings in quarter {quarter} for {ticker}")

    except Exception as e:
        print(f"Error fetching earnings for {ticker}: {e}")
        df.at[idx, "EarningsDate"] = None

print(df)


Error fetching earnings for AAPL: Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Error fetching earnings for AAPL: Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
Error fetching earnings for AAPL: Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
  Quarter Ticker EarningsDate
0   Q1-19   AAPL         None
1   Q1-19   AAPL         None
2   Q2-19   AAPL         None


In [18]:
import yfinance as yf
import pandas as pd

ticker = "AAPL"
quarter = "Q1-19"

# Helper to get quarter date range
def get_quarter_date_range(qtr_str):
    quarter_months = {
        "Q1": ("01-01", "03-31"),
        "Q2": ("04-01", "06-30"),
        "Q3": ("07-01", "09-30"),
        "Q4": ("10-01", "12-31"),
    }
    qtr, yr = qtr_str.split('-')
    year_full = int("20" + yr)
    start_date = pd.to_datetime(f"{year_full}-{quarter_months[qtr][0]}")
    end_date = pd.to_datetime(f"{year_full}-{quarter_months[qtr][1]}")
    return start_date, end_date

start_date, end_date = get_quarter_date_range(quarter)
print(f"Looking for earnings dates between {start_date.date()} and {end_date.date()}")

ticker_obj = yf.Ticker(ticker)

try:
    # Try get_earnings_dates
    earnings_dates_df = ticker_obj.get_earnings_dates(limit=10)
    print("Earnings Dates DataFrame:")
    print(earnings_dates_df)

    if earnings_dates_df is None or earnings_dates_df.empty:
        print("No earnings dates data found.")
    else:
        # Identify possible date columns
        date_cols = [col for col in earnings_dates_df.columns if 'date' in col.lower()]
        print("Date columns found:", date_cols)
        
        if date_cols:
            date_col = date_cols[0]
            earnings_dates_df[date_col] = pd.to_datetime(earnings_dates_df[date_col], errors='coerce')
            filtered = earnings_dates_df[
                (earnings_dates_df[date_col] >= start_date) & 
                (earnings_dates_df[date_col] <= end_date)
            ]
            print(f"Earnings dates within the quarter {quarter}:")
            print(filtered)
        else:
            print("No date columns found in earnings data.")
except Exception as e:
    print(f"Error fetching earnings dates: {e}")


Looking for earnings dates between 2019-01-01 and 2019-03-31
Earnings Dates DataFrame:
                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2026-04-29 06:59:00-04:00           NaN           NaN          NaN
2026-01-28 16:00:00-05:00           NaN           NaN          NaN
2025-10-29 06:59:00-04:00           NaN           NaN          NaN
2025-07-30 06:59:00-04:00           NaN           NaN          NaN
2025-05-01 16:30:00-04:00          1.63          1.65         1.41
2025-01-30 16:31:00-05:00          2.35          2.40         2.15
2024-10-31 16:31:00-04:00          1.60          1.64         2.35
2024-08-01 16:30:00-04:00          1.35          1.40         3.99
2024-05-02 16:31:00-04:00          1.50          1.53         1.97
2024-02-01 16:00:00-05:00          2.10          2.18         3.90
Date columns found: []
No date columns found in earnings data.


In [24]:
#changings
tickers = "WMT"
start_year = 2014
end_year = 2024

ticker = yf.Ticker(tickers)

# Historical Prices
price_data = ticker.history(start=f"{start_year}-01-01", end=f"{end_year}-12-31")

In [25]:
# Earnings Dates
earnings_df = ticker.get_earnings_dates(limit=100)
earnings_dates = earnings_df.loc[
    earnings_df.index.to_series().dt.year.between(start_year, end_year)
].index.normalize()

# Shares Outstanding
shares_outstanding = ticker.info.get("sharesOutstanding")

# --- 1) Market Cap on Earnings Dates ---
market_caps = []
for date in earnings_dates:
    date = pd.to_datetime(date) #normalize()

    # Shift to previous available trading day if it's not in the price data
    while date not in price_data.index:
        date -= pd.Timedelta(days=1)

    close_price = price_data.loc[date]['Close']
    market_cap = close_price * shares_outstanding
    market_caps.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Close Price': round(close_price, 2),
        'Estimated Market Cap': round(market_cap)
    })

market_cap_df = pd.DataFrame(market_caps)

# closing price df
closing_price_df = price_data[['Close']].copy()
closing_price_df.columns = [tickers]
closing_price_df.index = closing_price_df.index.normalize()
closing_price_df.index = closing_price_df.index.date

# --- 3) Latest Market Cap ---
latest_price = price_data['Close'].iloc[-1]
latest_market_cap = latest_price * shares_outstanding

info = ticker.info
sector = info.get("sector", "N/A")
industry = info.get("industry", "N/A")
long_name = info.get("longName", "N/A")
short_name = info.get("shortName", "N/A")

latest_market_cap_df = pd.DataFrame([{
    'Ticker': tickers,
    'Short Name': short_name,
    'Long Name': long_name,
    'Latest Price': round(latest_price, 2),
    'Latest Market Cap': round(latest_market_cap),
    'Sector': sector,
    'Industry': industry
}])

#Market Cap on Earnings Dates for singular ticker
print(market_cap_df)

#Daily Closing Prices - can add column for each ticker
print(closing_price_df.head(n=10))

#Last Market Caps - can add row for each ticker
latest_market_cap_df

          Date  Close Price  Estimated Market Cap
0   2024-11-19        85.96          687789142137
1   2024-08-15        72.44          579553715562
2   2024-05-16        63.36          506931258636
3   2024-02-20        57.63          461073039406
4   2023-11-16        50.94          407594740495
5   2023-08-17        50.83          406680576322
6   2023-05-18        49.28          394255885126
7   2023-02-21        47.55          380469378651
8   2022-11-15        47.41          379323468733
9   2022-08-16        44.82          358561546563
10  2022-05-17        42.05          336462836311
11  2022-02-17        44.13          353097407349
12  2021-11-16        45.31          362544777662
13  2021-08-17        47.70          381612816371
14  2021-05-18        44.75          358036707917
15  2021-02-18        43.06          344516832659
16  2020-11-17        46.55          372461652097
17  2020-08-18        41.98          335906225332
18  2020-05-19        38.78          310293513415


,Ticker,Short Name,Long Name,Latest Price,Latest Market Cap,Sector,Industry
0,WMT,Walmart Inc.,Walmart Inc.,90.1,720913126468,Consumer Defensive,Discount Stores
